Requirements
 - Input the data
 - Create a new column in the Part 1 Output table for each student’s initials. If a student has a double barreled second name then only take the first letter from the first part
 - e.g. “NERTY CHERRY HOLME” becomes NC not NCH. 
 - Find a way to join this table to the Additional Information table. We should maintain exactly 4,000 unique records. 
 - Develop a ranking system to rank each student by Grade Score within their specified Subject Selection and Region. Every combination of Subject Selection and Region should have their own ranking and remember that if students have a matching grade score, we then prioritise those who live closer to the school as a “tie-breaker”. 
 - For each Subject, find and flag the top 20 students with the caveat that this year within each course, 15 students must be from the East and 5 from the West given our newly imposed 75%/25% split
 - Delete all rejected students, leaving only the 100 accepted students.
 - Remove unnecessary fields
 - Find the total number of accepted applicants per secondary school and represent this as a percentage of the total spaces that were available for that region. 
 - Hint: think about how many were allowed per course, and how many courses there are
 - For each region, label their highest performing school as “High Performing” and the lowest performing school as “Low Performing” in a new column named “School Status”.
 - Give all other schools the status “Average Performing”
 - Delete any unwanted fields and rearrange to give the output shown below
 - Output the data
